# Process the data from Anna for the web

I will construct an sqlite database files for each cluster.

For now, I am only taking the members.  And I am planning to create a different database file for each cluster.  (Currently the sample here for NGC 6819 is ~1GB in size).

In [ ]:
import pandas as pd
import numpy as np
import csv
import os
import sqlite3

#from zipfile import ZipFile
import tarfile


In [ ]:
def process_cluster(cluster_name, prefix = '', 
                    run_cluster_res = True,
                    run_cluster_summary = True,
                    run_cluster_ms = True,
                    run_sample_mass_files = True,
                    ):
    print('processing files for : ', cluster_name)
    # create an sqlite file (or connect to existing file)
    conn = sqlite3.connect(os.path.join('sqlite', prefix, cluster_name + '.db'))
    cursor = conn.cursor()

    #with ZipFile(os.path.join('rawData', cluster_name + '.zip'), 'r') as zip_ref:
    with tarfile.open(os.path.join('rawData', prefix, cluster_name + '.tar.gz'), 'r:gz') as tar_ref:

        #file_list = zip_ref.namelist()
        file_list = tar_ref.getnames()

        if (run_cluster_res):
            # read in and process the cluster .res file
            #res_file = os.path.join(cluster_name, cluster_name + '.res')
            res_file = os.path.join('.', cluster_name + '.res')
            if res_file in file_list:
                #with zip_ref.open(res_file) as f:
                with tar_ref.extractfile(tar_ref.getmember(res_file)) as f:
                    print(res_file)
                    singlePop_res = pd.read_csv(f, delimiter = '\s+')
                    singlePop_res.insert(loc = 0, column = 'iteration', value = singlePop_res.index + 1)
                    singlePop_res.to_sql('cluster_posterior', conn, if_exists = 'replace', index = False)
            else:
                print('ERROR! did not find', res_file)


        if (run_cluster_summary or run_sample_mass_files):
            # read in and process the cluster summary file
            #sum_file = os.path.join(cluster_name, cluster_name + '.df')
            sum_file = os.path.join('.', cluster_name + '.df')
            if sum_file in file_list:
                #with zip_ref.open(sum_file) as f:
                with tar_ref.extractfile(tar_ref.getmember(sum_file)) as f:
                    print(sum_file)
                    stars_summary = pd.read_csv(f, delimiter = ' ')
                    stars_summary.drop('Unnamed: 0', axis = 1, inplace = True)
                    # looks like sqlite can't handle these as long ints
                    stars_summary['source_id'] = stars_summary['source_id'].astype(str)
                    # take only the members to reduce the file size?
                    stars_summary_members = stars_summary.loc[stars_summary['member']]
                    stars_summary_members.to_sql('stars_summary', conn, if_exists = 'replace', index = False)
            else:
                print('ERROR! did not find', sum_file)

        if (run_cluster_ms):
            # read in and process the .ms file
            #ms_file = os.path.join(cluster_name, cluster_name + '_dir.ms')
            ms_file = os.path.join('.', cluster_name + '.ms')
            if ms_file in file_list:
                #with zip_ref.open(ms_file) as f:
                with tar_ref.extractfile(tar_ref.getmember(ms_file)) as f:
                    print(ms_file)
                    ms_df = pd.read_csv(f, delimiter = '\s+')
                    # fix the column names because sqlite is case insensitive
                    ms_df.rename(columns = {'u':'u_SDSS', 'g':'g_SDSS', 'r':'r_SDSS', 'i':'i_SDSS', 'z':'z_SDSS'}, inplace = True)
                    ms_df.to_sql('parsec_isochrone', conn, if_exists = 'replace', index = False)
            else:
                print('ERROR! did not find', ms_file)

        if (run_sample_mass_files):
            # parse through the sampleMass files
            # get all the files
            # iterate through the files to read them all in, and create tables for each star
            print('sampleMass output ... ')
            memIDs = stars_summary_members['source_id'].to_numpy()
            have_sample_mass = np.array([False for x in memIDs])
            #sample_mass_files = [x for x in file_list if ('sampleMass.out' in x and '__MACOSX' not in x)]
            sample_mass_files = [x for x in file_list if ('samplemass.out' in x and '__MACOSX' not in x)]
            for i, smf in enumerate(sample_mass_files):
                #with zip_ref.open(smf) as f:
                with tar_ref.extractfile(tar_ref.getmember(smf)) as f:
                    print(f'{i} {(i+1)/len(sample_mass_files)*100:.2f} {smf}' )

                    df = pd.read_csv(f, delimiter = '\s+', quoting = csv.QUOTE_NONE)
                    df.rename(columns = {'starId':'source_id'}, inplace = True)
                    df['source_id'] = df['source_id'].astype(str)
                    try:
                        df['source_id'] = df['source_id'].str.replace('"', '', regex = True)
                    except:
                        pass

                    # get the unique IDs
                    uID = df['source_id'].unique()

                    for iden in uID:
                        # include only the members(?)
                        #if (np.int64(iden) in memIDs):
                        if (iden in memIDs):
                            # j = np.where(memIDs == np.int64(iden))[0]
                            j = np.where(memIDs == iden)[0]
                            have_sample_mass[j] = True
                            foo = df.loc[df['source_id'] == iden].copy()
                            foo.drop('source_id', axis = 1, inplace = True)
                            foo.to_sql('posterior_for_id_' + iden, conn, if_exists = 'replace', index = False)
                    
            found = np.where(have_sample_mass == True)[0]
            missing = np.where(have_sample_mass == False)[0]
            print('# of found sampleMass files : ', len(found))
            print('# of missing sampleMass files : ', len(missing))

    
    cursor.close()
    conn.close()

In [ ]:
# process the summary file

cluster_params = pd.read_csv(os.path.join('rawData','paper2','cluster_params.csv'))
cluster_params.columns = cluster_params.columns.str.replace(r"[()]", "")
cluster_params.columns = cluster_params.columns.str.replace(" ", "_")
cluster_params.sort_values(by='Cluster', inplace = True)


# add in NGC 6791 from the previous paper for the website, but not for zenodo
# from paper 1 we had completeness-corrected values, which I will drop here (with "_c")
# because of that, we had "_i" on some columns (for incomplete).  These are now our standard values, so I will remove the "_i".
cluster_params_paper1 = pd.read_csv(os.path.join('rawData','paper1','cluster_params.csv'))
cluster_params_paper1.columns = cluster_params_paper1.columns.str.replace(r"[()]", "")
cluster_params_paper1.columns = cluster_params_paper1.columns.str.replace(" ", "_")
cluster_6791 = cluster_params_paper1.loc[cluster_params_paper1['Cluster'] == 'NGC_6791'].reset_index(drop=True)
cluster_6791 = cluster_6791.loc[:,~cluster_6791.columns.str.endswith('_c')]
cluster_6791 = cluster_6791.loc[:,~cluster_6791.columns.str.contains('_c_')]
cluster_6791.columns = cluster_6791.columns.str.rstrip('_i') 
foo = cluster_6791['Center_RA_hr'].str.split()[0]
cluster_6791.loc[0,'Center_RA_hr'] = float(foo[0]) + float(foo[1])/60 + float(foo[2])/3600
foo = cluster_6791['Center_Dec_deg'].str.split()[0]
cluster_6791.loc[0,'Center_Dec_deg'] = float(foo[0]) + float(foo[1])/60 + float(foo[2])/3600
cluster_params_website = pd.concat([cluster_params, cluster_6791], ignore_index=True)

cluster_params_website.sort_values(by='Cluster', inplace = True)

cluster_params_website


## NOTE : the NGC 6791 data is missing some columns because Paper 1 differs slightly from Paper 2... I should note this in the README

In [ ]:
# convert to sql (for zenodo)
conn = sqlite3.connect(os.path.join('sqlite', 'paper2', 'cluster_summary.db'))
cursor = conn.cursor()
cluster_params.to_sql('cluster_parameters', conn, if_exists = 'replace', index = False)
cursor.close()
conn.close()

In [ ]:
# convert to sql (for website)
conn = sqlite3.connect(os.path.join('sqlite', 'paper2', 'cluster_summary_website.db'))
cursor = conn.cursor()
cluster_params_website.to_sql('cluster_parameters', conn, if_exists = 'replace', index = False)
cursor.close()
conn.close()

In [ ]:
cluster_params.columns

In [ ]:
# process the HDBSCAN file
# SHOULD I CHANGE no_points to num_objects or something? (wait for Anna's README info)
hdbscan_params = pd.read_csv(os.path.join('rawData','paper2','HDBSCAN_params.txt'), sep = '\s+')
hdbscan_params = hdbscan_params.rename(columns={'Min_size': 'min_size', 'Radius': 'radius', 'Group': 'group_number', 'no_points':'n_objects'})
hdbscan_params.sort_values(by='Cluster', inplace = True)

In [ ]:
hdbscan_params

In [ ]:

conn = sqlite3.connect(os.path.join('sqlite', 'paper2', 'cluster_summary.db'))
#conn = sqlite3.connect(os.path.join('sqlite', 'paper2', 'hdbscan_cluster_params.db'))
cursor = conn.cursor()
hdbscan_params.to_sql('hdbscan_cluster_parameters', conn, if_exists = 'replace', index = False)
cursor.close()
conn.close()

In [ ]:
conn = sqlite3.connect(os.path.join('sqlite', 'paper2', 'cluster_summary_website.db'))
cursor = conn.cursor()
hdbscan_params.to_sql('hdbscan_cluster_parameters', conn, if_exists = 'replace', index = False)
cursor.close()
conn.close()

In [ ]:
# paper 1
# clusters_to_process = ['NGC_6791','NGC_188','NGC_2682','NGC_6819','NGC_7789','NGC_2168']

# paper 2
# clusters_to_process = cluster_params['Cluster']


clusters_to_process = [
#    "Berkeley_32",
#    "Berkeley_39",
#    "Collinder_394",
#    "Haffner_22",
#    "Melotte_71",
#    "NGC_1245",
#    "NGC_1664",
#    "NGC_1817",
#    "NGC_188",
#    "NGC_1912",
#    "NGC_2099",
#    "NGC_2168",
#    "NGC_2281",
#    "NGC_2287",
#    "NGC_2301",
#    "NGC_2323",
#    "NGC_2355",
#    "NGC_2360",
#    "NGC_2420",
#    "NGC_2423",
#    "NGC_2437",
#    "NGC_2447",
#    "NGC_2506",
#    "NGC_2539",
#    "NGC_2548",
#    "NGC_2627",
#    "NGC_2682",
#    "NGC_6819",
#    "NGC_6940",
#    "NGC_7209",
#    "NGC_7243",
#    "NGC_7789",
#    "Ruprecht_171",
#    "Tombaugh_1",
#    "Trumpler_3",
]

for c in clusters_to_process:
    process_cluster(c, prefix = 'paper2')#,                   
                    # run_cluster_res = False,
                    # run_cluster_summary = False,
                    # run_cluster_ms = False,
                    # run_sample_mass_files = True)

# Testing accessing the data for the backend

In [ ]:
import sqlite3
import os

In [ ]:
data_dir = os.path.join(os.getcwd(), 'sqlite', 'paper2')
conn = sqlite3.connect(os.path.join(data_dir, 'NGC_7789.db'))
cursor = conn.cursor()

In [ ]:
def get_available_clusters():
    files = []
    clusters = []
    contents = os.listdir(data_dir)
    for item in contents:
        if os.path.isfile(os.path.join(data_dir, item)) and '.db' in item:
            files.append(os.path.join(data_dir, item))
            clusters.append(str.replace(item, '.db',''))

    return files, clusters
get_available_clusters()

In [ ]:
def get_available_tables(cursor):
    # get all the available tables
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tbls = cursor.fetchall()
    tables = [t[0] for t in tbls]
    return tables
get_available_tables(cursor)

In [ ]:
def get_available_columns(cursor, table_name):
    # Execute the PRAGMA to get table information
    cursor.execute(f"PRAGMA table_info({table_name})")

    # Fetch all rows of the result
    table_info = cursor.fetchall()

    # return the column names
    column_names = [row[1] for row in table_info]
    return column_names

get_available_columns(cursor, 'stars_summary')
#get_available_columns(cursor, 'cluster_posterior')
#get_available_columns(cursor, 'parsec_isochrone')
#get_available_columns(cursor, 'posterior_for_id_1995014851613762432')

In [ ]:
def get_column_data(cursor, table_name, column):
    # select the data from the table
    cursor.execute(f"SELECT {column} FROM {table_name}")
    
    # Fetch all the rows of that result
    dd = cursor.fetchall()
    
    # return the data
    data = [d[0] for d in dd]
    return data
get_column_data(cursor, 'posterior_for_id_1995014851613762432', 'membership')

In [ ]:
data_dir = os.path.join(os.getcwd(), 'sqlite', 'paper2')
conn = sqlite3.connect(os.path.join(data_dir, 'cluster_summary.db'))
cursor = conn.cursor()

In [ ]:
get_available_tables(cursor)

In [ ]:
get_available_columns(cursor, 'cluster_parameters')

In [ ]:
get_column_data(cursor, 'cluster_parameters', 'Age_Gyr')